In [196]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

In [2]:
data_file = "/home/scai/phd/aiz218323/scratch/Courses/nanoGPT/input.txt"

with open(data_file) as file:
    file_content = file.read()

In [5]:
print(file_content[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
characters = sorted(list(set(file_content)))
print(f"Number of characters : {len(characters)}")
print("".join(characters))

Number of characters : 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [10]:
stoi = {c:i for i, c in enumerate(characters)}
itos = {i:c for c, i in stoi.items()}

In [264]:
torch.randint(10, (1000,))

tensor([1, 5, 8, 0, 4, 8, 0, 2, 1, 4, 8, 0, 6, 1, 1, 9, 8, 1, 4, 0, 5, 2, 8, 1,
        8, 9, 7, 3, 1, 5, 5, 9, 9, 1, 7, 8, 4, 6, 7, 0, 8, 9, 7, 6, 6, 1, 8, 3,
        5, 0, 4, 6, 8, 0, 3, 3, 6, 8, 4, 7, 9, 7, 5, 0, 2, 7, 9, 1, 1, 0, 6, 7,
        4, 0, 6, 7, 0, 4, 8, 5, 7, 2, 8, 2, 8, 8, 4, 1, 0, 3, 9, 2, 9, 3, 1, 9,
        8, 8, 6, 3, 4, 8, 3, 1, 1, 0, 0, 3, 6, 7, 2, 9, 6, 3, 0, 2, 3, 3, 6, 1,
        7, 2, 8, 0, 3, 2, 0, 6, 9, 7, 0, 9, 7, 5, 8, 5, 6, 2, 9, 3, 6, 8, 0, 9,
        5, 2, 7, 5, 9, 0, 8, 5, 7, 2, 1, 3, 2, 9, 4, 1, 1, 3, 6, 1, 5, 5, 3, 3,
        3, 0, 5, 2, 0, 7, 3, 9, 3, 3, 4, 9, 1, 5, 1, 7, 8, 2, 3, 5, 4, 8, 2, 8,
        1, 5, 3, 0, 9, 6, 5, 4, 0, 8, 2, 7, 4, 4, 9, 5, 1, 9, 8, 5, 1, 4, 0, 6,
        5, 3, 1, 1, 6, 0, 9, 9, 9, 3, 3, 5, 2, 8, 2, 1, 3, 4, 4, 0, 8, 7, 9, 9,
        2, 4, 8, 6, 2, 3, 0, 7, 9, 9, 6, 7, 7, 9, 3, 9, 2, 3, 1, 4, 4, 8, 6, 0,
        5, 0, 2, 7, 9, 1, 0, 8, 7, 4, 0, 8, 3, 5, 4, 3, 2, 5, 9, 3, 4, 1, 1, 4,
        1, 5, 0, 6, 9, 2, 6, 9, 2, 2, 1,

In [270]:
def get_batch(block_size, batch_size):
    idxs = torch.randint(len(file_content)-block_size, (batch_size,))
    batch = torch.tensor([[stoi[c] for c in file_content[idx:idx+block_size+1]] for idx in idxs] , dtype=torch.long)
    x = batch[:, :block_size]
    y = batch[:, 1:block_size+1]
    return x, y

In [250]:
block_size = 8
batch_size = 4

x, y = get_batch(block_size, batch_size)

In [271]:
x.shape, y.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

Language modelling task

In [252]:
block_x = x[0]
block_y = y[0]

for t in range(1, example.shape[0]):
    print(f'{block_x[:t]} : {block_y[t]}')

tensor([58]) : 43
tensor([58, 46]) : 51
tensor([58, 46, 43]) : 1
tensor([58, 46, 43, 51]) : 59
tensor([58, 46, 43, 51,  1]) : 54
tensor([58, 46, 43, 51,  1, 59]) : 6
tensor([58, 46, 43, 51,  1, 59, 54]) : 0


In [272]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        self.co_occurence_matrix = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, x, y=None):
        logits = model.co_occurence_matrix(x)
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), y.contiguous().view(B*T))
        return logits, loss
    
    @torch.no_grad()
    def generate(self, max_length):
        s = "\n"
        for _ in range(max_length):
            logit, _ = self.forward(torch.tensor(stoi[s[-1]]))
            prob = F.softmax(logit, dim=0)
            idx = torch.multinomial(prob, num_samples=1)[0].item()
            s += itos[idx]
        return s

In [273]:
model = BigramLanguageModel(len(characters))
logits, loss = model(x, y)

In [274]:
print(model.generate(1000))


vDqz'AtkxLFdlDqSIhFyZl,!JkOTJiIU-mHbwok;n3t;Rkd!?LnRAsdgxnuO
BYTd;$tTv;rV'vATseVSxXX?cMMREr:q 3CijgLxpCTcM
j?sTXX-TZ-N?$PjLtE
r$vHb&k KbqMr.xoZoZ$ElsiC!qlV'fWKjxSSJGUfehfff?KfftLHEQ$LllioVSut'fuMtdk!q'CTiugnJ:XT,gUk;VD;H.xpehAlGDr$k NBYtKzC&?NCwygVDr,NjGQMmseV IuP?LFu e-ZyNVvS:vRMdUyJh&ks lfU,MLrE
BGMzoVh$kONj!i'f,mFxOhctLr,-Qu 3mBfFkCLS:j&?WoiHngQ$OATu
wRhkTh:fU:XM;DNcgl:fF xTHGDKbouYGX$kBCL;v-rD
aSSo:i.xoO&hK$k IQJra;lIwthpVljsNcMYtcygdrpTvLrOUyA3$tAFDeMmG?:DjUP SjwBCFxSjQm,-yMIEU'Dmoq?VVVtLWvGOo ?xvBY;NaJ&mOcDS:Ds!XYQ.
TfTuMzckxNj;aHT:NtRkmoRafwtMvUkOuc
:SusA G?D;yJ$d?bowzWKxAV,-EeXwYyorLrU:HTzueSu,kCPRH$Lr!FZtk vq?'a ,ff;uKTbqkeSijMb-SVDqqeJHe'3TsA;FKcDsA;IhhxTddGI:DiBrLrcGS.NsNIuYy'iC'MbB3
I$ytLrohKLO
AF?N:BrLGkJhP,GSuzJU

;UnH;RyAqJKNOw,UPhMvvy-DQ;MRfAjVqvATd-mGvd;UpeSC!
EuM;Hs Ef?'!Nr3PRh.?MVIDTNs KNaWKpgdQoZimCIMbqb-LS-;AXSuwxSMeQhjf?Vn,.wksin-SpA
GN;xAXd:jPYTboAKftkWKxPaM;fTA
F.wli:KlePlzoRJu,'.hpV'nFSRHboZo 3XBY,AE3WqO&xlQCLPhLZYKBk DJSMHDsLEDQdEW?wq'wiq S$3e
wkKlWVsjfTplruo

In [275]:
num_iterations = 10000
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [277]:
for i in range(num_iterations):
    x, y = get_batch(block_size, batch_size)
    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'Loss : {loss.item()}')

Loss : 2.2668299674987793


In [278]:
print(model.generate(1000))



ORD:
BForder mmand withavitohethinwinond cand w trth as?
A M:
thes e. t he pre blousther ing wak's f?
CLoveas tslir; am.-wor an,

Q!RAno
ARYONowainove neu.
Whemoust al an weis,
ANo d os, illlend the:

ARO:
Bu meppiato anchans, kelo spef
TINE:

KEThousho avieves tt, arud faiocharnonghand ldar f Whaco flvatorst ma ofacay
An wtur your, hthe.
Therd tousalitin S:

DQMInetoouise th co ilse ece oucheworcayoptheino s t.
O:
DYling ppiesa me.
G gee ohen s ws o the me't y leles hosevesit Soreacheckfuiu.

'd!
DWe th h iawired't;
Tonerind f-feo.
Fiswd way,
y ch,
NOUMall!vend tt f bom iediprte n ceje nond t her and hatl msmatoo cir yof uthiggrt sprd ainseanow healent thinghetrievery u, st ce,
Tin o-LEYo s ord celst theq-f wis FOFBurued it,

YO:
MKere IGOus t n mellllll de t, won! anthethericonowiverelicofosthe hldu mpellon Ifat
WI he dsig s fam be, wardve sbordyo whican.
'llst ll y he nourth d thanve hathis d t f ato prdng emiscund; Inckerestte bsty, wet se, fo buofo
I g

Thetick k mbil wnou?
Wis 